In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [2]:
import pandas as pd
import numpy as np
from bayesian.train_bn import structure_learning, parameter_learning, parameter_learning_mix, n_component
from preprocess.discretization import get_nodes_type, discretization, inverse_discretization, code_categories, get_nodes_sign
from bayesian.save_bn import save_structure, save_params, read_structure, read_params
from bayesian.sampling import generate_synthetics
from external.libpgm.hybayesiannetwork import HyBayesianNetwork
from visualization.visualization import draw_BN
from bayesian.calculate_accuracy import calculate_acc
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from copy import copy
from external.libpgm.sampleaggregator import SampleAggregator
import operator
from sklearn.metrics import accuracy_score, mean_squared_error
from scipy import stats
from scipy.stats import multivariate_normal
from sklearn.mixture import GaussianMixture
import math
from pomegranate import DiscreteDistribution
import random
from sklearn.model_selection import train_test_split

In [3]:
geo = geo = pd.read_csv('../data/hackathon_processed.csv')
columns = ['Tectonic regime', 'Period', 'Lithology', 'Structural setting', 'Hydrocarbon type', 'Gross','Netpay','Porosity','Permeability', 'Depth']
geo = geo[columns]
geo.dropna(inplace=True)
geo.reset_index(inplace=True, drop=True)

In [4]:
geo_types = get_nodes_type(geo)
geo_types

{'Tectonic regime': 'disc',
 'Period': 'disc',
 'Lithology': 'disc',
 'Structural setting': 'disc',
 'Hydrocarbon type': 'disc',
 'Gross': 'cont',
 'Netpay': 'cont',
 'Porosity': 'cont',
 'Permeability': 'cont',
 'Depth': 'cont'}

In [5]:
geo_signs = get_nodes_sign(geo)
geo_signs

{'Gross': 'pos',
 'Netpay': 'pos',
 'Porosity': 'pos',
 'Permeability': 'pos',
 'Depth': 'pos'}

In [6]:
colums_for_code = []
columns_for_disc = []
for c in columns:
    if geo_types[c] == 'disc':
        colums_for_code.append(c)
    else:
        columns_for_disc.append(c)

In [7]:
geo_coded, label_coder = code_categories(geo, 'label', colums_for_code)
geo_discrete, coder = discretization(geo_coded, 'equal_frequency', columns_for_disc)
geo_only_discrete, discrete_coder = discretization(geo, 'equal_frequency', columns_for_disc)

In [8]:
bn_full = structure_learning(geo_discrete, 'HC', geo_types, 'K2')
bn_full

  0%|          | 11/1000000 [00:01<36:30:52,  7.61it/s]


{'V': ['Tectonic regime',
  'Period',
  'Lithology',
  'Structural setting',
  'Hydrocarbon type',
  'Gross',
  'Netpay',
  'Porosity',
  'Permeability',
  'Depth'],
 'E': [['Tectonic regime', 'Structural setting'],
  ['Period', 'Porosity'],
  ['Period', 'Gross'],
  ['Lithology', 'Permeability'],
  ['Lithology', 'Gross'],
  ['Structural setting', 'Depth'],
  ['Structural setting', 'Gross'],
  ['Structural setting', 'Period'],
  ['Gross', 'Netpay'],
  ['Porosity', 'Permeability'],
  ['Porosity', 'Depth']]}

In [9]:
params = parameter_learning_mix(geo, geo_types, bn_full)
save_structure(bn_full, 'all_net')
skel = read_structure('all_net')
save_params(params, 'all_net_param')
params = read_params('all_net_param')
all_bn = HyBayesianNetwork(skel, params)

C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (8) found smaller than n_clusters (9). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  label = cluster.KMeans(n_clusters=self.n_components, n_init=1,
C:\Users\anaxa\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\mixture\_base.py:147: ConvergenceWarning:

In [10]:
acc, rmse, real, pred = calculate_acc(all_bn, geo, columns)

In [11]:
acc

{'Tectonic regime': 0.48,
 'Period': 0.37,
 'Lithology': 0.56,
 'Structural setting': 0.56,
 'Hydrocarbon type': 0.75}

In [12]:
rmse

{'Gross': 309.34,
 'Netpay': 80.89,
 'Porosity': 5.77,
 'Permeability': 925.91,
 'Depth': 856.08}

In [10]:
acc, rmse, real, pred = calculate_acc(all_bn, geo, columns)

In [11]:
rmse

{'Gross': 290.12,
 'Netpay': 81.06,
 'Porosity': 5.75,
 'Permeability': 925.48,
 'Depth': 854.5}

In [11]:
acc, rmse, real, pred = calculate_acc(all_bn, geo, columns)

54


In [12]:
rmse

{'Gross': 289.91,
 'Netpay': 72.99,
 'Porosity': 5.75,
 'Permeability': 815.91,
 'Depth': 836.21}

In [8]:
accuracy_dict = dict()
rmse_dict = dict()
pred_param = [[0 for j in range(geo.shape[0])] for i in range(len(columns))]
real_param = [[0 for j in range(geo.shape[0])] for i in range(len(columns))]
for i in range (geo.shape[0]):
    test = dict(geo.iloc[i,:])
    train_data = geo_discrete.drop(index=i)
    param_train = geo.drop(index=i)
    train_data.reset_index(inplace=True, drop = True)
    param_train.reset_index(inplace=True, drop = True)
    bn = structure_learning(train_data, 'HC', geo_types, 'K2')
    params = parameter_learning(param_train, geo_types, bn)
    save_structure(bn, 'all_net')
    skel = read_structure('all_net')
    save_params(params, 'all_net_param')
    params = read_params('all_net_param')
    all_bn = HyBayesianNetwork(skel, params)
    for n, key in enumerate(columns):
        train_dict = copy(test)
        train_dict.pop(key)
        try:
            sample = generate_synthetics(all_bn, geo_signs, evidence=train_dict)
            if sample.shape[0] == 0:
                continue
            else:
                if geo_types[key] == 'disc':
                    dict_top_probs = dict()
                    probs = dict(sample.groupby(key)[key].count() / sample.shape[0])
                    sorted_res = sorted(probs.items(), key=operator.itemgetter(1), reverse=True)
                    pred_param[n][i] = sorted_res[0][0]
                    real_param[n][i] = test[key]
                if geo_types[key] == 'cont':
                    pred_param[n][i] = np.mean(sample[key].values)
                    real_param[n][i] = test[key]
        except:
            continue
for n, key in enumerate(columns):
        if geo_types[key] == 'disc':
            accuracy_dict[key] = round(accuracy_score(real_param[n], pred_param[n]),2)
        if geo_types[key] == 'cont':
            rmse_dict[key] = round(mean_squared_error(real_param[n], pred_param[n], squared=False),2)

|          | 11/1000000 [00:01<47:36:34,  5.83it/s]
  0%|          | 13/1000000 [00:02<56:17:13,  4.93it/s]


In [9]:
accuracy_dict

{'Tectonic regime': 0.48,
 'Period': 0.31,
 'Lithology': 0.56,
 'Structural setting': 0.56,
 'Hydrocarbon type': 0.75}

In [10]:
rmse_dict

{'Gross': 351.58,
 'Netpay': 82.64,
 'Porosity': 6.0,
 'Permeability': 971.99,
 'Depth': 927.07}

In [8]:
accuracy_dict = dict()
rmse_dict = dict()
pred_param = [[0 for j in range(geo.shape[0])] for i in range(len(columns))]
real_param = [[0 for j in range(geo.shape[0])] for i in range(len(columns))]
for i in range (geo.shape[0]):
    test = dict(geo.iloc[i,:])
    train_data = geo_discrete.drop(index=i)
    param_train = geo.drop(index=i)
    train_data.reset_index(inplace=True, drop = True)
    param_train.reset_index(inplace=True, drop = True)
    bn = structure_learning(train_data, 'HC', geo_types, 'K2')
    params = parameter_learning_mix(param_train, geo_types, bn)
    save_structure(bn, 'all_net')
    skel = read_structure('all_net')
    save_params(params, 'all_net_param')
    params = read_params('all_net_param')
    all_bn = HyBayesianNetwork(skel, params)
    for n, key in enumerate(columns):
        train_dict = copy(test)
        train_dict.pop(key)
        try:
            sample = generate_synthetics(all_bn, geo_signs, evidence=train_dict)
            if sample.shape[0] == 0:
                continue
            else:
                if geo_types[key] == 'disc':
                    dict_top_probs = dict()
                    probs = dict(sample.groupby(key)[key].count() / sample.shape[0])
                    sorted_res = sorted(probs.items(), key=operator.itemgetter(1), reverse=True)
                    pred_param[n][i] = sorted_res[0][0]
                    real_param[n][i] = test[key]
                if geo_types[key] == 'cont':
                    pred_param[n][i] = np.mean(sample[key].values)
                    real_param[n][i] = test[key]
        except:
            continue
for n, key in enumerate(columns):
        if geo_types[key] == 'disc':
            accuracy_dict[key] = round(accuracy_score(real_param[n], pred_param[n]),2)
        if geo_types[key] == 'cont':
            rmse_dict[key] = round(mean_squared_error(real_param[n], pred_param[n], squared=False),2)

t/s]
  0%|          | 13/1000000 [00:02<53:48:09,  5.16it/s]


In [9]:
accuracy_dict

{'Tectonic regime': 0.48,
 'Period': 0.3,
 'Lithology': 0.56,
 'Structural setting': 0.56,
 'Hydrocarbon type': 0.75}

In [10]:
rmse_dict

{'Gross': 332.77,
 'Netpay': 82.58,
 'Porosity': 6.0,
 'Permeability': 973.09,
 'Depth': 926.29}

In [20]:
import numpy as np
from gmr import GMM

# Your dataset as a NumPy array of shape (n_samples, n_features):
X = np.random.randn(100, 3)

gmm = GMM(n_components=3, random_state=0)
gmm.from_samples(X)

# Make predictions with known values for the first feature:
x1 = np.random.randn(1, 2).tolist()
x1_index = [0, 1]
x2_predicted_mean = gmm.predict(x1_index, x1)

In [23]:
x2_predicted_mean[0][0]

-0.3595376587117654